### 신경망 모델 구성하기
* 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성됨.
* torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공함.
* PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass)임.
* 신경망은 다른 모듈(계층:layer)로 구성된 모듈임.
* 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있음.

* FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성해봄.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

### 학습을 위한 장치 얻기

* 가능한 경우 GPU 또는 MPS 와 같은 하드웨어 가속기에서 모델을 학습하려고 함.
* torch.cuda 또는 torch.backends.mps 가 사용 가능한지 확인해보고 그렇지 않으면 CPU를 계속 사용함.

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### 클래스 정의하기
* 신경망 모델을 nn.Module 의 하위클래스로 정의하고, __init__ 에서 신경망 계층들을 초기화함.
* nn.Module 을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현함.

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

* NeuralNetwork 의 인스턴스(instance)를 생성하고 이를 device로 이동한 뒤, 구조(structure)를 출력함.

In [10]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


* 모델을 사용하기 위해 입력 데이터를 전달함.
* 이는 일부 백그라운드 연산들과 함께 모델의 forward 를 실행함.

* 모델에 입력을 전달하여 호출하면 2차원 텐서를 반환함.
* 2차원 텐서의 dim=0은 각 분류(class)에 대한 원시(raw) 예측값 10개가, dim=1에는 각 출력의 개별 값들이 해당함.
* 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻음.

In [11]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


### 모델 계층(Layer)
* FashionMNIST 모델의 계층들을 살펴봄.
* 이를 설명하기 위해, 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 살펴봄.

In [12]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
* nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환함. (dim=0의 미니배치 차원은 유지됨.)

In [13]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
* 선형 계층은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈.

In [14]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
* 비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듦.
* 비선형 활성화는 선형 변환 후에 적용되어 비선형성(nonlinearity)을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 도움.

* 이 모델에서는 nn.ReLU 를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있음.

In [15]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2003,  0.0921, -0.0877,  0.4008,  0.4457,  0.1930,  0.3111,  0.6718,
         -0.0387, -0.3068,  0.1017,  0.3880, -0.1016,  0.1790, -0.1417, -0.1472,
         -0.2718, -0.0951, -0.1711, -0.2424],
        [-0.1635,  0.3707,  0.0665,  0.2304,  0.2108,  0.2132, -0.0461,  0.5223,
         -0.0629, -0.1568, -0.4342,  0.3236, -0.0273,  0.1875, -0.1903,  0.0052,
         -0.0315, -0.3024, -0.2208, -0.5194],
        [-0.1097,  0.2308,  0.2566,  0.4442,  0.5248,  0.3454, -0.0820,  0.5673,
          0.1387, -0.0050, -0.3689,  0.3668, -0.0760,  0.0646, -0.0232,  0.0739,
         -0.3617, -0.2392, -0.1208, -0.0965]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0921, 0.0000, 0.4008, 0.4457, 0.1930, 0.3111, 0.6718, 0.0000,
         0.0000, 0.1017, 0.3880, 0.0000, 0.1790, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.3707, 0.0665, 0.2304, 0.2108, 0.2132, 0.0000, 0.5223, 0.0000,
         0.0000, 0.0000, 0.3236, 0.0000, 0.1875, 0.00

### nn.Sequential
* nn.Sequential 은 순서를 갖는 모듈의 컨테이너.
* 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됨.
* 순차 컨테이너(sequential container)를 사용하여 아래의 seq_modules 와 같은 신경망을 빠르게 만들 수 있음.

In [17]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

### nn.Softmax
* 신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits를 반환함.
* logits 는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)됨.
* dim 매개변수는 값의 합이 1이 되는 차원을 나타냄.

In [18]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### 모델 매개변수
* 신경망 내부의 많은 계층들은 매개변수화(parameterize)됨.
* 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어짐.
* nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 parameters() 및 named_parameters() 메소드로 모든 매개변수에 접근할 수 있게 됨.

* 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력함.

In [19]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0062,  0.0127, -0.0016,  ...,  0.0266,  0.0342, -0.0076],
        [-0.0208,  0.0329, -0.0195,  ...,  0.0123,  0.0198,  0.0077]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0044, -0.0259], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0132,  0.0323, -0.0282,  ...,  0.0380, -0.0357,  0.0350],
        [ 0.0030,  0.0266,  0.0427,  ..., -0.0200, -0.0023, -0.0288]],
       device='cuda:0', grad_fn=<Slice